<a href="https://colab.research.google.com/github/CodingYodha/QnA-bot-using-GRU/blob/main/QnA_with_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/CodingYodha/QnA-bot-using-GRU.git